In [2]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, KFold
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import load_model
from sklearn.metrics import confusion_matrix, mean_squared_error, accuracy_score
import tensorflow as tf
import joblib

# Set seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Create directories if they don't exist
graphs_dir = 'graphs_nn_diabetes'
models_dir = 'models_nn_diabetes'
os.makedirs(graphs_dir, exist_ok=True)
os.makedirs(models_dir, exist_ok=True)

# Load the dataset
data_path = 'diabetes_binary_5050split.csv'
data = pd.read_csv(data_path)

# Separate features and target
X = data.drop('Diabetes_binary', axis=1)  # Adjust the column name if necessary
y = data['Diabetes_binary']  # Adjust the column name if necessary

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

# Standardize the data
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# Save the scaler
joblib.dump(scaler, os.path.join(models_dir, 'scaler.joblib'))

# Define the model creation function
def create_model(activation='relu', learning_rate=0.001):
    model = Sequential()
    model.add(Dense(64, input_dim=X_train.shape[1], activation=activation))
    model.add(Dense(64, activation=activation))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])
    return model

# Hyperparameters
activations = ['relu', 'tanh']
learning_rates = [0.01, 0.001]
kfold = KFold(n_splits=5, shuffle=True, random_state=42)

# Train the models with cross-validation
history_logs = []
global_suffix_count = 1

train_accuracies_dict = {activation: {lr: [] for lr in learning_rates} for activation in activations}
val_accuracies_dict = {activation: {lr: [] for lr in learning_rates} for activation in activations}

for activation in activations:
    for learning_rate in learning_rates:
        best_val_accuracy = 0
        best_epoch = 0
        fold_no = 1
        
        for train_index, val_index in kfold.split(X_train):
            X_train_fold, X_val_fold = X_train[train_index], X_train[val_index]
            y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

            model = create_model(activation=activation, learning_rate=learning_rate)
            history = model.fit(X_train_fold, y_train_fold, epochs=100, batch_size=32, validation_data=(X_val_fold, y_val_fold), verbose=0, callbacks=[EarlyStopping(monitor='val_loss', patience=10)])

            for epoch, val_acc in enumerate(history.history['val_accuracy']):
                if val_acc > best_val_accuracy:
                    best_val_accuracy = val_acc
                    best_epoch = epoch + 1

            train_accuracies_dict[activation][learning_rate].extend(history.history['accuracy'])
            val_accuracies_dict[activation][learning_rate].extend(history.history['val_accuracy'])

            fold_no += 1

        # Collect history logs
        history_logs.append((activation, learning_rate, best_val_accuracy, best_epoch))

        # Save the model
        model_path = os.path.join(models_dir, f'model_{activation}_{learning_rate}_best.h5')
        model.save(model_path)

        global_suffix_count += 1

# Plot combined learning curves
for activation in activations:
    plt.figure()
    for learning_rate in learning_rates:
        plt.plot(train_accuracies_dict[activation][learning_rate], label=f'Training accuracy (lr={learning_rate})')
        plt.plot(val_accuracies_dict[activation][learning_rate], label=f'Validation accuracy (lr={learning_rate})', linestyle='--')
    plt.title(f'Learning Curves for activation={activation}')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend(loc='best')
    plt.grid()
    learning_curve_path = os.path.join(graphs_dir, f'combined_learning_curve_{activation}.png')
    plt.savefig(learning_curve_path)
    plt.close()

# Evaluate the best models on the test set and generate combined validation curves
suffix_count = global_suffix_count
val_accuracies_combined = {activation: {lr: [] for lr in learning_rates} for activation in activations}
best_val_accuracies = {activation: [] for activation in activations}

for activation in activations:
    for learning_rate in learning_rates:
        model_path = os.path.join(models_dir, f'model_{activation}_{learning_rate}_best.h5')
        model = load_model(model_path)
        
        # Recompile the model to reset the optimizer
        model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=learning_rate), metrics=['accuracy'])

        history = model.fit(X_train, y_train, epochs=best_epoch, batch_size=32, validation_data=(X_test, y_test), verbose=0)
        
        best_val_accuracies[activation].append(history.history['val_accuracy'][-1])

# Plot combined validation curves
plt.figure()
for activation in activations:
    plt.plot(learning_rates, best_val_accuracies[activation], label=f'Validation accuracy ({activation})', marker='o')
plt.title('Validation Curves for Different Learning Rates')
plt.xlabel('Learning Rate')
plt.ylabel('Validation Accuracy')
plt.xscale('log')
plt.legend(loc='best')
plt.grid()
validation_curve_path = os.path.join(graphs_dir, 'combined_validation_curve.png')
plt.savefig(validation_curve_path)
plt.close()

# Generate confusion matrix for the best model
best_model_info = max(history_logs, key=lambda x: x[2])
best_activation, best_learning_rate, best_val_accuracy, best_epoch = best_model_info
best_model_path = os.path.join(models_dir, f'model_{best_activation}_{best_learning_rate}_best.h5')
best_model = load_model(best_model_path)

# Recompile the best model to reset the optimizer
best_model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=best_learning_rate), metrics=['accuracy'])

# Predict on the test set
y_pred = best_model.predict(X_test)
y_pred_classes = (y_pred > 0.5).astype(int)

# Confusion Matrix
conf_matrix = confusion_matrix(y_test, y_pred_classes)

# Normalize the confusion matrix by row (i.e by the number of samples in each class)
conf_matrix_normalized = conf_matrix.astype('float') / conf_matrix.sum(axis=1)[:, np.newaxis] * 100

# Plot the normalized confusion matrix
plt.figure(figsize=(10, 7))
sns.heatmap(conf_matrix_normalized, annot=True, fmt='.2f', cmap='Blues')
plt.title('Confusion Matrix for Best Model (Percentage)')
plt.xlabel('Predicted')
plt.ylabel('True')
conf_matrix_path = os.path.join(graphs_dir, f'confusion_matrix_best_model_percentage.png')
plt.savefig(conf_matrix_path)
plt.close()

print("All tasks completed successfully. Models and graphs are saved in their respective directories.")


c:\Users\chinu\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\chinu\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)
c:\Users\chinu\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in

663/663 ━━━━━━━━━━━━━━━━━━━━ 0s 340us/step
All tasks completed successfully. Models and graphs are saved in their respective directories.
